# Введение

В данной практике, мы сделаем геодатасет для опытов с библиотеками `S2` & `H3`. Сравним данные библиотеки. К сожалению, сравнение будет не совсем корректным, так как `S2` написана на `C++` и официального порта на `python` - нет. Поэтому будем использовать [s2sphere](https://github.com/sidewalklabs/s2sphere), которая содержит необходимый минимум библиотеки `S2`.

В данной задаче мы рассмотрим, как с помощью данных библиотек легко можно решить задачу **"point-in-polygon"**. То есть когда нам нужно для определенной области определить какие из точек входят в нужные нам "нарисованные" полигоны. Тем самым можно агрегировать точечные данные и легко проводить дальнейший анализ.

Если быть точнее, то мы спарсим данные о домах для одного уездного города нашей страны (Ноябрьска, ЯНАО) с сайта `dom.mingkh.ru`. Зная количество домов и количество квартир, в каждом из них. Мы косвенно получим численность населения данного города, перемножив количество квартир на средней состав семьи = 3,504 (папа + мама + [суммарный коэфициент рождаемости](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D1%81%D1%82%D1%80%D0%B0%D0%BD_%D0%BF%D0%BE_%D1%81%D1%83%D0%BC%D0%BC%D0%B0%D1%80%D0%BD%D0%BE%D0%BC%D1%83_%D0%BA%D0%BE%D1%8D%D1%84%D1%84%D0%B8%D1%86%D0%B8%D0%B5%D0%BD%D1%82%D1%83_%D1%80%D0%BE%D0%B6%D0%B4%D0%B0%D0%B5%D0%BC%D0%BE%D1%81%D1%82%D0%B8)), распределенное в пространстве или границах города. Тем самым можно выявить точки наибольшей плотности населения.

Конечно, подобный тип данных можно найти в интернете, или с помощью парсинга других сайтов. Но для учебных целей будем брать данные с сайта [dom.mingkh.ru](https://dom.mingkh.ru/)

**UPD** К сожелению, сайт dom.mingkh.ru защитился от парсинга с помощью cloudflare, когда делал курс защиты не было =(( А обойти такую защиту не так просто, так как курс не про парсинг/скрапинг,то лучше пропустить данный пункт и промотать до S2. Там есть сохраненные данные и оттуда можно смело все запускать. Весь код парсинга закомментировал

# Создадим свой геодатасет для опытов

Из-за структуры данного сайта нам придется дважды его парсить, сначала узнаем количество домов, и их адреса. Они нам понадобятся для обратного геокодинга, чтобы получить координаты домов.

И вторым заходом будем заходить на страничку каждого дома и брать данные о количестве квартир в нем.

Чтобы не ждать и не запускать ячейки парсинга, можно сразу перейти к следующим частям. Так как парсинг занимает около получаса.

 ## Парсим список домов с dom.mingkh.ru

Установка необходимых библиотек

In [ ]:
# !pip install -q selenium beautifulsoup4 webdriver-manager google-colab-selenium[undetected]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


Устанавливаем драйвер браузера Chrome

In [ ]:
# !apt update
# !apt install chromium-chromedriver

Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import geopandas as gpd
import re
# from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException

In [ ]:
# # Инициализируем Chrome драйвер
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome(options=options)

# # Создаем пустой df
# columns = ['index', 'city', 'address', 'square', 'year', 'floor', 'link']
# df = pd.DataFrame(columns=columns)

# # Определяем функцию для парсинга html-страницы
# def html_to_df(df, page_source):

#     # Используем BeautifulSoup для парсинга HTML контента
#     soup = BeautifulSoup(page_source, 'html.parser')

#     # Ищем все строки таблицы:
#     rows = soup.find_all('tr')

#     data = []
#     for row in rows:
#         # Вытаскиваем все данные из ячеек таблицы
#         # на самом деле нужен только адрес и гиперссылка на дом, но вдруг
#         # в будущем пригодяться и эти данные
#         cells = row.find_all('td')
#         row_data = [cell.text.strip() for cell in cells]

#         # Вытаскиваем дополнительно гиперссылку на каждый дом, она нам еще понадобится
#         link = row.find('a')
#         href = link['href'] if link else None
#         row_data.append(href)

#         data.append(row_data)

#     # Создаем времянный датафрейм и соединяем с прошлым датафреймом
#     df_temp = pd.DataFrame(data, columns=df.columns)
#     df = pd.concat([df,df_temp])

#     return df

# # Открываем первую web страницу
# driver.get("https://dom.mingkh.ru/yamalo-neneckiy-ao/noyabrsk/houses")

# # Ждем 10 сек для загрузки страницы, зависит от контента страницы
# driver.implicitly_wait(10)

# # "Сохраняем исходник страницы" и передаем ее для парсинга
# page_source = driver.page_source
# df = html_to_df(df, page_source)

# # Открываем следующие страницы
# while True:
#     try:
#         next_page_button = driver.find_element(By.XPATH, "//a[@rel='next']")
#         next_page_button.click()
#         # "Сохраняем исходник страницы" и передаем ее для парсинга
#         page_source = driver.page_source
#         df = html_to_df(df, page_source)
#     except NoSuchElementException:
#         print("Next page not found. Finish")
#         break

# # Закрываем драйвер
# driver.quit()

Next page not found. Finish


Посмотрим, что мы на парсили

In [ ]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    0 non-null      object
 1   city     0 non-null      object
 2   address  0 non-null      object
 3   square   0 non-null      object
 4   year     0 non-null      object
 5   floor    0 non-null      object
 6   link     0 non-null      object
dtypes: object(7)
memory usage: 124.0+ bytes


In [ ]:
# df.isna().sum()

index      11
city       11
address    11
square     11
year       11
floor      11
link       11
dtype: int64

11 лишних строк, по количеству страниц, это шапка таблицы. Удалим их

In [ ]:
# df = df.dropna()
# df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068 entries, 1 to 68
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    1068 non-null   object
 1   city     1068 non-null   object
 2   address  1068 non-null   object
 3   square   1068 non-null   object
 4   year     1068 non-null   object
 5   floor    1068 non-null   object
 6   link     1068 non-null   object
dtypes: object(7)
memory usage: 66.8+ KB


In [ ]:
# df.head()

,index,city,address,square,year,floor,link
1,1,Ноябрьск,"ул. 40 лет Победы, 3",4589.4,1987,5,/yamalo-neneckiy-ao/noyabrsk/509296
2,2,Ноябрьск,"ул. 40 лет Победы, 7б",4074.4,1987,5,/yamalo-neneckiy-ao/noyabrsk/509299
3,3,Ноябрьск,"ул. 40 лет Победы, 7",66.9,1987,5,/yamalo-neneckiy-ao/noyabrsk/509297
4,4,Ноябрьск,"ул. 40 лет Победы, 7в",3584,1987,5,/yamalo-neneckiy-ao/noyabrsk/509300
5,5,Ноябрьск,"ул. 40 лет Победы, 7А",3198.8,1987,5,/yamalo-neneckiy-ao/noyabrsk/509298


Сохраним и перейдем к следующему этапу

In [ ]:
# df.to_csv('houses_Noyabrsk.csv', index=False)

## Парсим количество квартир в каждом доме с dom.mingkh.ru

Теперь спарсим количество квартир в каждом доме

In [ ]:
# def parsing_count_floors(link):
#     # Инициализируем Chrome драйвер
#     options = webdriver.ChromeOptions()
#     options.add_argument('--headless')
#     options.add_argument('--no-sandbox')
#     options.add_argument('--disable-dev-shm-usage')
#     driver = webdriver.Chrome(options=options)

#     # Открываем web страницу дома
#     driver.get("https://dom.mingkh.ru" + link)

#     # Получаем исходник страницы и закрываем драйвер
#     page_source = driver.page_source
#     driver.quit()

#     # Парсим страницу с помощью BeautifulSoup
#     soup = BeautifulSoup(page_source, 'html.parser')

#     # Ищем количество жилых помещений
#     floor_count_text = soup.find(string=re.compile("Жилых помещений"))
#     if floor_count_text:
#         parent = floor_count_text.parent
#         if parent:
#             floor_count = parent.find_next_sibling()
#             if floor_count:
#                 count = floor_count.text.strip()
#     else:
#         # если частный дом или нет данных, то жилое помещение 1
#         count = 1

#     return count

Следующая команда выполняется минут 35, так как парситься 1000+ страниц. Удалим лишнюю колонку с индексом и сохраним

In [ ]:
# df['floors_count'] = df['link'].apply(parsing_count_floors)
# df = df.drop(['index'], axis=1)
# df.to_csv('houses_Noyabrsk.csv', index=False)

Чтобы не ждать полчаса, можно загрузить готовый файл с помощью следующей команды

In [ ]:
# !gdown 1a0OaqN2k8bUdSGdHumRuoNP5a8ELJFGj

Downloading...
From: https://drive.google.com/uc?id=1a0OaqN2k8bUdSGdHumRuoNP5a8ELJFGj
To: /content/houses_Noyabrsk.csv
100% 111k/111k [00:00<00:00, 60.6MB/s]


In [ ]:
# df = pd.read_csv('/content/houses_Noyabrsk.csv')

Проверим, что у нас тоже самое

In [ ]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          1068 non-null   object 
 1   address       1068 non-null   object 
 2   square        1068 non-null   float64
 3   year          1068 non-null   object 
 4   floor         1068 non-null   object 
 5   link          1068 non-null   object 
 6   floors_count  1068 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 58.5+ KB


In [ ]:
# df['floors_count'].value_counts()

1      535
4       68
2       64
60      43
16      29
      ... 
404      1
100      1
77       1
155      1
68       1
Name: floors_count, Length: 94, dtype: int64

Очень много данных не хватает((( Так как одноквартирных домов очень много, примерно половина (а это не правда, в данном городе в основном 5-этажки). Что поделать такой источник данных =(

In [ ]:
# Считаем количество людей
# df['count_people'] = df['floors_count'] * 3.504
# df['count_people'] = df['count_people'].astype('int64')

И проверим на сколько мы ошиблись в расчетах

In [ ]:
# df['count_people'].sum()

71950

Примерно 30 тысяч людей потеряно (если верить официальной статистике в данном городе около 100к людей проживает).
Мне кажется это отличным **примером** достоверности и доверия к данным при парсинге. Никогда об этом не забывайте!!!

In [ ]:
# df.to_csv('houses_Noyabrsk_2.csv', index=False)

## Геокодинг

Чтобы окончательно сформировать датасет, нужно добавить координаты. Займемся этим с помощью `Nominatium`. Но сначала сделаем колонку полного адреса.

In [ ]:
# !gdown 1EdAKTmmFPT9trlVi15topxKeGzIke7x5

Downloading...
From: https://drive.google.com/uc?id=1EdAKTmmFPT9trlVi15topxKeGzIke7x5
To: /content/houses_Noyabrsk_2.csv
100% 113k/113k [00:00<00:00, 2.98MB/s]


In [ ]:
# df = pd.read_csv('/content/houses_Noyabrsk_2.csv')

In [ ]:
# df['full_addr'] = df['city'] + ' ' + df['address']
# df = df.drop(['city', 'address'], axis=1)

Удалим все запятые в адресе. Не знаю почему, но без них ищется лучше в `openstreet`

In [ ]:
# df['full_addr'] = df['full_addr'].apply(lambda x: x.replace(',', ''))

In [ ]:
# df.head()

,square,year,floor,link,floors_count,count_people,full_addr
0,4589.4,1987,5,/yamalo-neneckiy-ao/noyabrsk/509296,89,311,Ноябрьск ул. 40 лет Победы 3
1,4074.4,1987,5,/yamalo-neneckiy-ao/noyabrsk/509299,75,262,Ноябрьск ул. 40 лет Победы 7б
2,66.9,1987,5,/yamalo-neneckiy-ao/noyabrsk/509297,135,473,Ноябрьск ул. 40 лет Победы 7
3,3584.0,1987,5,/yamalo-neneckiy-ao/noyabrsk/509300,50,175,Ноябрьск ул. 40 лет Победы 7в
4,3198.8,1987,5,/yamalo-neneckiy-ao/noyabrsk/509298,60,210,Ноябрьск ул. 40 лет Победы 7А


In [ ]:
# geocoded_df = gpd.tools.geocode(
#     # указываем колонку с адресами для геокодинга
#     df["full_addr"],
#     # Инициализируем Nominatium
#     provider="nominatim",
#     user_agent="geopyExample",
#     timeout=3
# )
# geocoded_df.head()

/usr/local/lib/python3.10/dist-packages/geopandas/geoseries.py:751: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()


,geometry,address
0,POINT (75.44871 63.20261),"3, улица 40 лет Победы, Ноябрьск, городской ок..."
1,POINT (75.44696 63.20380),"7Б, улица 40 лет Победы, Ноябрьск, городской о..."
2,POINT (75.44834 63.20424),"7, улица 40 лет Победы, Ноябрьск, городской ок..."
3,POINT (75.44682 63.20452),"7В, улица 40 лет Победы, Ноябрьск, городской о..."
4,POINT (75.44598 63.20414),"7А, улица 40 лет Победы, Ноябрьск, городской о..."


Естественно, что адреса не все нашлись. Проверим сколько у нас пропусков

In [ ]:
# geocoded_df['address'].isna().sum()

472

Далее соединим два датасета, найдем список пустых и пропустим через бесплатный геокодинг Яндекса, как раз в лимит уложимся.

In [ ]:
# geocoded_df = geocoded_df.join(df)

In [ ]:
# series_to_geocode = geocoded_df.loc[geocoded_df['address'].isna(), 'full_addr']

In [ ]:
# geocoded_df_2 = gpd.tools.geocode(
#     # указываем колонку с адресами для геокодинга
#     series_to_geocode,
#     # Инициализируем Yandex
#     provider="Yandex",
#     api_key='YOUR_API_KEY'
# )
# geocoded_df_2.head()

,geometry,address
32,POINT (77.00204 62.98179),"улица 70 лет Октября, 2, микрорайон Вынгапуров..."
33,POINT (77.00280 62.98169),"улица 70 лет Октября, 4, микрорайон Вынгапуров..."
34,POINT (77.00286 62.98218),"улица 70 лет Октября, 6, микрорайон Вынгапуров..."
35,POINT (77.00393 62.98224),"улица 70 лет Октября, 8, микрорайон Вынгапуров..."
36,POINT (77.00371 62.98256),"улица 70 лет Октября, 10, микрорайон Вынгапуро..."


Смотрим сколько пропусков снова

In [ ]:
# geocoded_df_2['address'].isna().sum()

0

Яндекс - найдется все! Шутка =) Продолжаем делать наш геодатасет.

In [ ]:
# geocoded_df.loc[geocoded_df['address'].isna(), ['geometry', 'address']] = geocoded_df_2

Проверяем

In [ ]:
# geocoded_df['address'].isna().sum()

0

Приведем в порядок и сохраним наш геодатасет

In [ ]:
# geocoded_df = (geocoded_df
#               .drop(['full_addr', 'link'], axis=1)
#               .reindex(columns=['address', 'square', 'year', 'floor',
#                                 'floors_count', 'count_people', 'geometry'])
#               )
# geocoded_df.head()

,address,square,year,floor,floors_count,count_people,geometry
0,"3, улица 40 лет Победы, Ноябрьск, городской ок...",4589.4,1987,5,89,311,POINT (75.44871 63.20261)
1,"7Б, улица 40 лет Победы, Ноябрьск, городской о...",4074.4,1987,5,75,262,POINT (75.44696 63.20380)
2,"7, улица 40 лет Победы, Ноябрьск, городской ок...",66.9,1987,5,135,473,POINT (75.44834 63.20424)
3,"7В, улица 40 лет Победы, Ноябрьск, городской о...",3584.0,1987,5,50,175,POINT (75.44682 63.20452)
4,"7А, улица 40 лет Победы, Ноябрьск, городской о...",3198.8,1987,5,60,210,POINT (75.44598 63.20414)


In [ ]:
# geocoded_df.to_file("houses_Noyabrsk.gpkg", driver="GPKG")

Теперь можно перейти к нашему сравнению

# S2

Для тех кто промотал парсинг, сделали геодатасет с координатами домов и количестве людей, живущих в них, для города Ноябрьск. Получившийся датасет, не отображает реальной картины, так как около 30к жителей потеряно, из-за не полноты данных на сайте, с которого парсили данные.

Теперь попробуем сагрегировать данные с помощью библиотеки [s2sphere](https://github.com/sidewalklabs/s2sphere) и отобразить на карте.

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
!gdown 1xa8xUunsf45l-_PM9T_udi-2IhMLOhZQ

In [ ]:
gdf = gpd.read_file('/content/houses_Noyabrsk.gpkg')

In [ ]:
gdf.head()

,address,square,year,floor,floors_count,count_people,geometry
0,"3, улица 40 лет Победы, Ноябрьск, городской ок...",4589.4,1987,5,89,311,POINT (75.44871 63.20261)
1,"7Б, улица 40 лет Победы, Ноябрьск, городской о...",4074.4,1987,5,75,262,POINT (75.44696 63.20380)
2,"7, улица 40 лет Победы, Ноябрьск, городской ок...",66.9,1987,5,135,473,POINT (75.44834 63.20424)
3,"7В, улица 40 лет Победы, Ноябрьск, городской о...",3584.0,1987,5,50,175,POINT (75.44682 63.20452)
4,"7А, улица 40 лет Победы, Ноябрьск, городской о...",3198.8,1987,5,60,210,POINT (75.44598 63.20414)


Установим библиотеки

In [ ]:
!pip -q install s2sphere folium

Импортируем библиотеку и библиотеку для карт

In [ ]:
import s2sphere as s2
import folium
import branca.colormap as cm

Напомню задачу, нужно поверх города нарисовать сеть и раскрассить ее согласно численности населения. Где плотность выше - цвета будут темнее, где плотность ниже там светлее.

Чтобы решить данную задачу нужно пройти несколько шагов:
1. Определить область которую хотим покрыть сеткой.
2. Покрыть сеткой данную область.
3. Присвоить каждой точке датасета id ячейки из нарисованной сетки.
4. Сагрегировать данные о количество для кажжой ячейки.
5. Перевести ячейки в формат GeoJSON, так проще отобразить на карте
6. Добавить к ячейкам GeoJSON значения населения
7. Сделать цветовую шкалу, согласно значению населения
8. Отобразить карту.

Поехали...

Шаг 1

In [ ]:
# Задаем региион в форме прямоугольника,
# который мы хотим покрыть ячейками
p1 = s2.LatLng.from_degrees(63.2251, 75.3887)
p2 = s2.LatLng.from_degrees(63.1729, 75.5923)
region = s2.LatLngRect.from_point_pair(p1, p2)

Шаг 2

In [ ]:
# Покрываем выбранный регион сеткой ячеек
def create_s2_grid(region, level):
    coverer = s2.RegionCoverer()
    # Для нашего случая интересней покрыть регион однородной сеткой,
    # поэтому мин и макс уровень одинаковый.
    # Если нужно нужно создать сетку из "больших" и "маленьких" ячеек,
    # то нужно задать разные уровни.
    coverer.min_level = level
    coverer.max_level = level
    coverer.max_cells = 500

    covering = coverer.get_covering(region)
    cells = [s2.Cell(cell_id) for cell_id in covering]

    return cells

# 15 уровень - это ячейка примерно 250м на 290м
# Более подробно можно посмотреть тут - http://s2geometry.io/resources/s2cell_statistics
s2_cells = create_s2_grid(region, level=15)

Шаг 3

In [ ]:
def assign_s2_cell_id(gdf, level):
  # Назначаем каждой точке датасета соответствующий id ячейки, в котором она
  # находится согласно своим координатам
    def get_cell_id(lat, lon):
        latlng = s2.LatLng.from_degrees(lat, lon)
        cell = s2.CellId.from_lat_lng(latlng).parent(level)

        return cell.id()

    # Формируем новый столбец для каждой точки
    gdf['s2_cell_id'] = gdf.apply(lambda row: get_cell_id(row['geometry'].y, row['geometry'].x), axis=1)

    return gdf

# Необходимо выставить тот же уровень для сетки, что использовали выше
gdf = assign_s2_cell_id(gdf, level=15)

Шаг 4

In [ ]:
# Группируем по cell ID и суммируем количество человек
aggregated_data = gdf.groupby('s2_cell_id').agg({'count_people': 'sum'}).reset_index()

Шаг 5

In [ ]:
def s2_cells_to_geojson(cells):
    features = []

    for cell in cells:
        vertices = []
        # ищем координаты каждой вершины квадратов ячеек
        for i in range(4):
            vertex = cell.get_vertex(i)
            latlng = s2.LatLng.from_point(vertex)
            vertices.append([latlng.lng().degrees, latlng.lat().degrees])
        # Закрываем полигон, добавив первую вершину в конец
        vertices.append(vertices[0])

        # Добавляем в свойства полигона токен S2
        # токен - это буквенное представление 64-го id ячейки
        features.append({
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [vertices]
            },
            "properties": {
                "s2_token": cell.id().to_token()
            }
        })

    return {"type": "FeatureCollection", "features": features}

s2_grid_geojson = s2_cells_to_geojson(s2_cells)

Шаг 6

In [ ]:
def map_values_to_s2_cells(cells, aggregated_data):
    value_map = pd.Series(aggregated_data['count_people'].values, index=aggregated_data['s2_cell_id']).to_dict()

    # Добавляем новое свойсто для каждой ячейки - количество людей
    for feature in cells['features']:
        cell_id = s2.CellId.from_token(feature['properties']['s2_token'])
        feature['properties']['count_people'] = value_map.get(cell_id.id(), 0)

    return cells

# Создаем сетку S2 в GeoJSON формате с количеством людей
s2_grid_geojson = map_values_to_s2_cells(s2_grid_geojson, aggregated_data)

Шаг 7

Задаем функцию для создания цветовой шкалы, которая сопоставляет агрегированные значения с цветами, причем более высокие значения соответствуют более темным цветам.

In [ ]:
def get_color_scale(aggregated_data):
    max_value = aggregated_data['count_people'].max()
    # цветовая шкала будет в синих тонах
    color_scale = cm.linear.Blues_09.scale(0, max_value)

    return color_scale

color_scale = get_color_scale(aggregated_data)

Шаг 8

Используем `Folium` для отображения полигонов на карте с созданной цветовой схемой.

In [ ]:
def create_colored_map(s2_grid_geojson, color_scale):
    # Задаем центр карты и начальный зум
    m = folium.Map(location=[63.199096, 75.468806], zoom_start=15)

    folium.GeoJson(
        # Передаем нашу сеть и цветовую шкалу
        s2_grid_geojson,
        style_function=lambda feature: {
            'fillColor': color_scale(feature['properties']['count_people']),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        }
    ).add_to(m)

    return m

# Создаем и отображаем карту
m = create_colored_map(s2_grid_geojson, color_scale)
m

Вот таким образом можно отобразить сеть S2 на выбранной области и сагрегировать данные. Конечно, достоверность построенной карты вызывает сомнения из-за не полноты данных, но в общих чертах соотевствует действительности.

Теперь зная распределение плотности населения можно использовать эту информацию для открытия кафе, к примеру, или другого заведения.

Еще пару полезных ссылок о `S2`:
* [Онлайн визуализация S2 сетки](https://igorgatis.github.io/ws2/). Можно выбрать регион, отобразить на нем сетку и скачать данную сеть в формате `GeoJSON`
* [Справочная информация по уровням сетки](https://s2geometry.io/resources/s2cell_statistics)
* [s2cell](https://docs.s2cell.aliddell.com/en/stable/api/s2cell.s2cell.html). Минимальная реализация `S2` на python.

# H3

Теперь проделаем тоже самое при помощи библиотеки H3

In [ ]:
!pip -q install h3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.7 MB/s eta 0:00:00


In [ ]:
from h3 import h3
from shapely.geometry import Polygon

После того как установили, будем действовать по тому же принципу, каждой строчке присвоем номер ячейки h3, далее сагрегируем данные и отобразим ячейки на карте.

In [ ]:
# Масштаб возьмем 9, он примерно соответствует нашей ячейки s2
h3_level = 9

def lat_lng_to_h3(row):
    return h3.geo_to_h3(row.geometry.y,
                        row.geometry.x,
                        h3_level)

# Присваиваем номер ячейки h3
gdf['h3'] = gdf.apply(lat_lng_to_h3, axis=1)

Полную таблицу масштабов можно посмотреть [тут](https://h3geo.org/docs/core-library/restable)

In [ ]:
# Агрегируем данные о численности
aggregated_data_2 = gdf.groupby('h3').agg({'count_people': 'sum'}).reset_index()

In [ ]:
def add_geometry(row):
    points = h3.h3_to_geo_boundary(row['h3'], True)

    return Polygon(points)

# По номерам ячеек получаем их координаты и формируем столбец геометрии
aggregated_data_2['geometry'] = aggregated_data_2.apply(add_geometry, axis=1)

Рисуем карту

In [ ]:
# Задаем центр карты и начальный зум
m = folium.Map(location=[63.199096, 75.468806], zoom_start=14)

# Задаем цветовую шкалу и присваеваем цвет для каждой строки
max_count = aggregated_data_2['count_people'].max()
colormap = cm.linear.Blues_09.scale(0, max_count)
aggregated_data_2['color'] = aggregated_data_2['count_people'].apply(lambda x: colormap(x))

# Добавляем наши гексагоны к карте
for _, row in aggregated_data_2.iterrows():
    folium.GeoJson(data=row['geometry'],
                    style_function=lambda x, color=row['color']: {
                        'fillColor': color,
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.6
                        }
                   ).add_to(m)

# Добавляем подложку и выводим карту
colormap.add_to(m)
m

Получили такой же резултат, на много быстрее и легче.

# Вывод или итого

Вот таким образом с помощью двух рассмотренных библиотек можно "быстро" обощить данные основываясь на местоположении объектов и их характеристиках.